# coastdown_pipeline

**Short Introduction**  
The `coastdown_pipeline` function analyzes **coastdown (neutral-gear) segments** to **estimate drag parameters** via a **quadratic model**. It optionally performs **pitch correction**, **time-domain smoothing**, **segment trimming**, and **data subsampling**. Finally, it computes **aerodynamic drag** and **rolling resistance** coefficients, along with losses for various suspension levels. This pipeline is used for **RLC (f0, f1, f2) estimation**, which helps characterize vehicle drag characteristics over different speeds.

---

## Parameters

- **data_cache** (`dict` or `None`)  
  A dictionary containing pre-loaded data keyed by filenames. If `None`, the pipeline reads data from disk using `read_can_data`.

- **files** (`str` or `list` of `str`)  
  Accepts either:  
  1. A **folder path** (string) from which to gather files using `get_can_files`, or  
  2. A **list of file paths** to process individually.

- **do_pitch_correction** (`bool`, default=False)  
  If `True`, invokes `pitch_correction(...)` to remove pitch-induced forces (`mass*g*sin(pitch)`) from the measured deceleration.

- **speed_signal** (`str`)  
  Name of the speed column (e.g., `'vehicle_speed'` or `'vehicle_speed_gps'`) for binning and aggregating deceleration data.

- **bucket_size** (`float`)  
  The speed “window size” (in km/h) for grouping data into buckets (e.g., 5 km/h, 10 km/h).

- **vehicle_mass** (`float`, default=2300)  
  Overall vehicle mass (in kg). Used for calculating deceleration forces and drag coefficients.

- **rotating_mass_eq** (`float`, default=50)  
  Approximate mass (in kg) representing additional rotating inertia (wheels, driveshaft, etc.).

- **frontal_area** (`float`, default=2.3)  
  Vehicle frontal area (in m²). Used with `vehicle_mass` to compute aerodynamic coefficients.

- **smoothing_kwargs** (`dict`, optional)  
  Arguments for **time-domain smoothing** of specified columns (`columns_to_smooth`).  
  For example:  
  - `{'filter_type': 'moving_average', 'window_size': 3}`  
  - `{'filter_type': 'exponential_moving_average', 'alpha': 0.4}`  
  - `{'filter_type': 'savitzky_golay', 'window_size': 5, 'polyorder': 2}`  
  - `{'filter_type': 'lowpass', 'cutoff_frequency': 0.1, 'order': 2}`  

- **columns_to_smooth** (`list`, optional)  
  Columns to be smoothed in time. If `'vehicle_speed'` is included, it will be replaced internally with `speed_signal`.

- **steering_angle_limit** (`float` or `None`)  
  If provided, excludes data where `steering_wheel_angle` exceeds ±`steering_angle_limit`.

- **select_suspension_level** (`int` or `None`)  
  If given, discards coastdown segments that do not match this `suspension_level` value.

- **deriv_lower_limit**, **deriv_upper_limit** (`float` or `None`)  
  If both are provided, filters out points not within the allowed range of speed derivative (e.g., acceleration/deceleration bounds).

- **cut_time** (`float` or `None`)  
  Removes this many seconds at the start and end of each segment to avoid transient effects (if specified).

- **seed** (`int` or `None`)  
  If set, used alongside `target_n_segments` to randomly subsample the data.

- **target_n_segments** (`int` or `None`)  
  If set (with a valid `seed`), randomly selects the desired number of segments from the full dataset.

- **outlier_threshold** (`float` or `None`)  
  If provided, points that deviate beyond this threshold in the **quadratic fitting** step are removed as outliers.

- **loss_type** (`str`, default='rel')  
  Determines which loss metric is computed in `calculate_all_losses`. Possible values include:  
  - `'rel'` (relative error)  
  - `'abs'` (absolute)  
  - `'sec_rmse'` (RMSE within a specified section)

- **generate_plots** (`bool`, default=False)  
  If `True`, creates plots of raw and processed coastdown segments, plus final braking-force fits.

- **verbose** (`bool`, default=False)  
  If `True`, prints additional progress logs and outcomes.

- **filename_prefix** (`str`, default='coastdown')  
  Prefix for saved figure filenames (e.g., `coastdown_segments_processed.pdf`).

- **figsize_1**, **figsize_2** (`tuple`, default=(10, 5))  
  Figure sizes for segment overview plots and for the final braking-force fit plots, respectively.

- **speed_range** (`tuple` or `None`)  
  If specified, applies a speed range (e.g., `(15, 160)`) for fitting and plotting.

---

## Return Values

A **20-element tuple** comprising:

1. **`f0`**, **`f1`**, **`f2`**  
   Quadratic coastdown parameters from a model of the form  
   \[
       F(v) = f_0 + f_1 \cdot v + f_2 \cdot v^2
   \]

2. **`c_air`**, **`c_rr`**  
   Aerodynamic drag coefficient and rolling-resistance coefficient, derived from `(f1, f2)` and vehicle properties.

3. **`metric_rm_steering_angle`**  
   Number of data points removed due to steering angle filtering.

4. **`metric_rm_brake_pedal`**  
   Number of data points removed because the brake pedal was activated.

5. **`metric_rm_suspension_level`**  
   Number of data points removed for non-matching suspension level.

6. **`metric_rm_cut_time`**  
   Number of data points trimmed away from the start/end of each segment.

7. **`metric_rm_sampling`**  
   Points removed if a random sample of segments (`target_n_segments`) was taken.

8. **`total_removed`**  
   Sum of all removed data points (steering angle, brake, suspension level, time-window cuts, and sampling).

9. **`metric_outliers`**  
   Points excluded during the outlier removal process in the quadratic fit.

10. **`loss_very_low_suspension`, `loss_low_suspension`, `loss_medium_suspension`, `loss_high_suspension`**  
    Calculated losses under different assumed suspension levels.

11. **`loss_CoC_values`, `loss_EPA_values`**  
    Additional or regulatory comparison losses (e.g., CoC or EPA methodology).

12. **`lowest_loss_name`**  
    A string key (e.g., `'very_low_suspension'`) identifying the smallest loss among the computed values.

13. **`elapsed_time`**  
    Elapsed time (in seconds) for the entire pipeline.

In [ ]:
from modules.parametric_pipelines import coastdown_pipeline
from modules.data_handler import get_can_files

path = 'data/coastdown_data'
files = get_can_files(folder=path)

# Set parameters for the coastdown pipeline
do_pitch_correction = True
speed_signal = 'vehicle_speed_pitch_corrected'
bucket_size = 6
vehicle_mass = 2300
rotating_mass_eq = 50
frontal_area = 2.33
smoothing_kwargs = {'filter_type': 'exponential_moving_average', 'alpha': 0.05}
columns_to_smooth = ['accelerator_pedal', 'steering_wheel_angle', speed_signal]
steering_angle_limit = 10
select_suspension_level = None
# 2.5 = Lift = high_suspension
# 2 = Mittel = medium_suspension
# 1.5 = Abgesenkt = low_suspension
# 1 = Tief = very_low_suspension
deriv_lower_limit = -0.3
deriv_upper_limit = 0
seed = 42
target_n_segments = None
outlier_threshold = None
cut_time = 5  # seconds
loss_type = 'sec_rmse'
generate_plots = True
verbose = False
filename_prefix = 'filename_prefix'  # Adjust as needed
figsize_1 = (6, 3)
figsize_2 = (6, 3)
speed_range = (0, 210)

f0_pc, f1_pc, f2_pc, c_air, c_rr, metric_rm_steering_angle, metric_rm_brake_pedal, metric_rm_suspension_level, metric_rm_cut_time, metric_rm_sampling, total_removed, metric_outliers, loss_very_low_suspension, loss_low_suspension, loss_medium_suspension, loss_high_suspension, loss_CoC_values, loss_EPA_values, lowest_loss_name, elapsed_time, force_df_pc, v_model_pc, force_model_pc = coastdown_pipeline(
    data_cache=None,
    files=files,
    do_pitch_correction=do_pitch_correction,
    speed_signal=speed_signal,
    bucket_size=bucket_size,
    vehicle_mass=vehicle_mass,
    rotating_mass_eq=rotating_mass_eq,
    frontal_area=frontal_area,
    smoothing_kwargs=smoothing_kwargs,
    columns_to_smooth=columns_to_smooth,
    steering_angle_limit=steering_angle_limit,
    select_suspension_level=select_suspension_level,
    deriv_lower_limit=deriv_lower_limit,
    deriv_upper_limit=deriv_upper_limit,
    cut_time=cut_time,
    seed=seed,
    target_n_segments=target_n_segments,
    outlier_threshold=outlier_threshold,
    loss_type=loss_type,
    generate_plots=generate_plots,
    verbose=verbose,
    filename_prefix=filename_prefix,
    figsize_1=figsize_1,
    figsize_2=figsize_2,
    speed_range=speed_range
)

# Print results and round to 3 decimal places
print(f'f0: {f0_pc:.3f}')
print(f'f1: {f1_pc:.3f}')
print(f'f2: {f2_pc:.3f}')
print(f'c_air: {c_air:.3f}')
print(f'c_rr: {c_rr:.3f}')
print(f'Filtered out total of {total_removed} data points')
print(f'Metric - Removed due to steering angle: {metric_rm_steering_angle}')
print(f'Metric - Removed due to brake pedal: {metric_rm_brake_pedal}')
print(f'Metric - Removed due to suspension level: {metric_rm_suspension_level}')
print(f'Metric - Removed due to time window cut: {metric_rm_cut_time}')
print(f'Metric - Removed due to sampling: {metric_rm_sampling}')
print(f'Metric - Removed as outliers: {metric_outliers}')
print(f'Elapsed time: {elapsed_time:.2f} seconds')

print(f'Loss against very low suspension: {loss_very_low_suspension:.3f}')
print(f'Loss against low suspension: {loss_low_suspension:.3f}')
print(f'Loss against medium suspension: {loss_medium_suspension:.3f}')
print(f'Loss against high suspension: {loss_high_suspension:.3f}')
print(f'Loss CoC values: {loss_CoC_values}')
print(f'Loss EPA values: {loss_EPA_values}')
print(f'Lowest loss name: {lowest_loss_name}')